In [1]:
import csv, sqlite3
import pandas as pd
con = sqlite3.connect("DB_py_sql.db")
cur = con.cursor()

In [2]:
#Magic module for SQL
%load_ext sql

df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCensusData.csv")
df.to_sql("CENSUS_DATA", con, if_exists='replace', index=False, chunksize=500)
print("CENSUS_DATA DONE")

df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv")
df.to_sql("CHICAGO_PUBLIC_SCHOOLS", con, if_exists='replace', index=False, chunksize=500)
print("CHICAGO_PUBLIC_SCHOOLS DONE")

df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCrimeData.csv")
df.to_sql("CHICAGO_CRIME_DATA", con, if_exists='replace', index=False, chunksize=500)
print("CHICAGO_CRIME_DATA DONE")

CENSUS_DATA DONE
CHICAGO_PUBLIC_SCHOOLS DONE
CHICAGO_CRIME_DATA DONE


### Problem

##### Find the total number of crimes recorded in the CRIME table.

In [3]:
%sql sqlite:///DB_py_sql.db
%sql SELECT COUNT(DISTINCT ID) FROM CHICAGO_CRIME_DATA;

 * sqlite:///DB_py_sql.db
Done.


COUNT(DISTINCT ID)
533


### Problem

##### List community area names and numbers with per capita income less than 11000.

In [4]:
%%sql 
SELECT COMMUNITY_AREA_NAME, COMMUNITY_AREA_NUMBER 
FROM CENSUS_DATA 
WHERE PER_CAPITA_INCOME <11000

 * sqlite:///DB_py_sql.db
Done.


COMMUNITY_AREA_NAME,COMMUNITY_AREA_NUMBER
West Garfield Park,26.0
South Lawndale,30.0
Fuller Park,37.0
Riverdale,54.0


### Problem

##### List all kidnapping crimes involving a child?

In [5]:
%%sql 
SELECT *
FROM CHICAGO_CRIME_DATA 
WHERE PRIMARY_TYPE = 'KIDNAPPING' AND 
    (LOWER(DESCRIPTION) LIKE '%child%' OR 
    LOWER(DESCRIPTION) LIKE '%child' OR
    LOWER(DESCRIPTION) LIKE 'child%');

 * sqlite:///DB_py_sql.db
Done.


ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,BEAT,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
5276766,HN144152,2007-01-26,050XX W VAN BUREN ST,1792,KIDNAPPING,CHILD ABDUCTION/STRANGER,STREET,0,0,1533,15,29.0,25.0,20,1143050.0,1897546.0,2007,41.87490841,-87.75024931,"(41.874908413, -87.750249307)"


### Problem

##### List the type of schools along with the average safety score for each type.

In [6]:
%%sql
SELECT "Elementary, Middle, or High School", ROUND(AVG(SAFETY_SCORE),2) AS AVERAGE 
FROM CHICAGO_PUBLIC_SCHOOLS
GROUP BY "Elementary, Middle, or High School"

 * sqlite:///DB_py_sql.db
Done.


"Elementary, Middle, or High School",AVERAGE
ES,49.52
HS,49.62
MS,48.0


### Problem

##### Which community area is most crime prone? Display the coumminty area number only.

In [7]:
%%sql 
SELECT COMMUNITY_AREA_NUMBER
FROM CHICAGO_CRIME_DATA 
GROUP BY COMMUNITY_AREA_NUMBER
ORDER BY COUNT(COMMUNITY_AREA_NUMBER) DESC
LIMIT 1

 * sqlite:///DB_py_sql.db
Done.


COMMUNITY_AREA_NUMBER
25.0


### Problem

##### Use a sub-query to find the name of the community area with highest hardship index

In [8]:
%%sql
SELECT COMMUNITY_AREA_NUMBER, COMMUNITY_AREA_NAME
FROM CENSUS_DATA 
WHERE hardship_index = (SELECT MAX(hardship_index) FROM CENSUS_DATA)

 * sqlite:///DB_py_sql.db
Done.


COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME
54.0,Riverdale


### Problem

##### Use a sub-query to determine the Community Area Name with most number of crimes?

In [9]:
%%sql 
SELECT CR.COMMUNITY_AREA_NUMBER, CD.COMMUNITY_AREA_NAME
FROM CHICAGO_CRIME_DATA CR, CENSUS_DATA CD
WHERE CR.COMMUNITY_AREA_NUMBER = CD.COMMUNITY_AREA_NUMBER
AND CR.COMMUNITY_AREA_NUMBER = (SELECT COMMUNITY_AREA_NUMBER
                                FROM CHICAGO_CRIME_DATA 
                                GROUP BY COMMUNITY_AREA_NUMBER
                                ORDER BY COUNT(COMMUNITY_AREA_NUMBER) DESC
                                LIMIT 1)
GROUP BY CR.COMMUNITY_AREA_NUMBER, CD.COMMUNITY_AREA_NAME

 * sqlite:///DB_py_sql.db
Done.


COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME
25.0,Austin
